MobileNet: Designed for mobile and embedded vision applications, it is efficient in terms of speed and memory usage.

Step 1: Set constant values

In [2]:
DATA_FILEPATH = ".\\dataset_26"
IMAGE_SIZE = (256, 256)
SEED = 2837373

Step 2: Load and Preprocess Data

In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

def load_data(folder_path):
    data = []
    for img in os.listdir(folder_path):
        try:
            img_path = os.path.join(folder_path, img)
            img = Image.open(img_path).convert('RGB')
            img = img.resize(IMAGE_SIZE)
            img_array = np.array(img)
            data.append(img_array)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
    return np.array(data)


data_list = []
labels_list = []
categories = ['notumor', 'glioma', 'meningioma', 'pituitary']
for category in categories:
    data_temp = load_data(DATA_FILEPATH + "\\" + category)
    labels_temp = np.full(len(data_temp), categories.index(category))

    data_list.extend(data_temp)
    labels_list.extend(labels_temp)

data = np.array(data_list)
labels = np.array(labels_list)

x_train, x_validate, y_train, y_validate= train_test_split(data, labels, train_size = 0.80, random_state = SEED, stratify=labels)  
x_train, x_test, y_train, y_test= train_test_split(x_train, y_train, train_size = 0.875, random_state = SEED, stratify=y_train)   

Step 3: Build & Compile the Model

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

input_shape = x_train.shape[1:]

# Load the MobileNet model pre-trained on ImageNet, excluding the top layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 128, 128, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 128, 128, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 128, 128, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 128, 128, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 128, 128, 32)      12

Step 4: Train the Model

In [5]:
tf.random.set_seed(SEED)
model.fit(x_train, y_train, epochs=10, validation_data=(x_validate, y_validate))

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[:50]:
    layer.trainable = False
for layer in base_model.layers[50:]:
    layer.trainable = True

# Re-compile the model after unfreezing layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Continue training (fine-tuning)
model.fit(x_train, y_train, epochs=10, validation_data=(x_validate, y_validate))

Epoch 1/10


11/11 [==============================] - 23s 1s/step - loss: 2.0236 - accuracy: 0.4375 - val_loss: 0.9164 - val_accuracy: 0.6875
Epoch 2/10
11/11 [==============================] - 11s 1s/step - loss: 0.6099 - accuracy: 0.7262 - val_loss: 0.6912 - val_accuracy: 0.7917
Epoch 3/10
11/11 [==============================] - 11s 990ms/step - loss: 0.4503 - accuracy: 0.8125 - val_loss: 0.5687 - val_accuracy: 0.7708
Epoch 4/10
11/11 [==============================] - 11s 1s/step - loss: 0.3024 - accuracy: 0.8958 - val_loss: 0.5249 - val_accuracy: 0.7917
Epoch 5/10
11/11 [==============================] - 11s 1s/step - loss: 0.2064 - accuracy: 0.9345 - val_loss: 0.5202 - val_accuracy: 0.8021
Epoch 6/10
11/11 [==============================] - 11s 1s/step - loss: 0.1882 - accuracy: 0.9494 - val_loss: 0.4934 - val_accuracy: 0.8021
Epoch 7/10
11/11 [==============================] - 11s 978ms/step - loss: 0.1399 - accuracy: 0.9702 - val_loss: 0.5195 - val_accuracy: 0.7812
Epoch 8/10
1

Step 5: Evaluate the Model

In [6]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

2/2 - 1s - loss: 0.4867 - accuracy: 0.8750 - 1s/epoch - 630ms/step

Test accuracy: 0.875


Step 6: Graphs